In [142]:
from pathlib import Path
import pandas as pd

data_root = Path("../kkdata3")
for x in data_root.glob("*"):
    print(x)

train_target = pd.read_parquet(data_root / "label_train_target.parquet")
test_source = pd.read_parquet(data_root / "label_test_source.parquet")
meta_song = pd.read_parquet(data_root / "meta_song.parquet")

../kkdata3/predict_result.parquet
../kkdata3/label_train_source.parquet
../kkdata3/label_train_target.parquet
../kkdata3/label_test_source.parquet
../kkdata3/meta_song_titletext.parquet
../kkdata3/sample.csv
../kkdata3/meta_song_lyricist.parquet
../kkdata3/meta_song_producer.parquet
../kkdata3/meta_song.parquet
../kkdata3/meta_song_genre.parquet
../kkdata3/meta_song_composer.parquet


In [143]:
train_source = pd.read_parquet(data_root / "label_train_source.parquet")
train_source.sort_values(["session_id", "listening_order"], inplace=True)

In [144]:
# def cal_score(df):

In [145]:
# calculate ngram frequency.
# n=1 is the most frequent of song_id
# n=2 is the most frequent of [song_id and next1_song_id], get the result from next2_song_id column
def getFreq(df, n=1):
    # calculate ngram frequency
    df_freq = (
        df.groupby(["song_id"] + [f"next{i}_song_id" for i in range(1, n)])
        .value_counts(sort=True, normalize=True)
        .reset_index(name="freq")
    )

    # get the most frequent song_id, sort by freq
    df_freq = (
        df_freq.sort_values(
            ["song_id"] + [f"next{i}_song_id" for i in range(1, n)] + ["freq"],
            ascending=False,
        )
        .groupby(["song_id"] + [f"next{i}_song_id" for i in range(1, n)])
        .head(1)
    )
    return df_freq


def getTrainData(df, n=2):
    new_df = df.copy()
    # gen n song id be the dataset
    for i in range(1, n + 1):
        new_df[f"next{i}_song_id"] = new_df["song_id"].shift(-i)

    # check if last song id is in the same session
    new_df[f"next{n}_session_id"] = new_df["session_id"].shift(-n)
    new_df = new_df.query(f"session_id == next{i}_session_id")

    # only get the song_id and next1_song_id, next2_song_id, next3_song_id... column
    new_df = new_df[["song_id"] + [f"next{i}_song_id" for i in range(1, n + 1)]]

    return new_df

In [146]:
tow_gram = getTrainData(train_source, 2)
tow_gram.head(20)

,song_id,next1_song_id,next2_song_id
10952316,f6f06a71bb8bc38af6c0b7dae9cab00d,7b48a87effd31c9c07b68ed212062854,61c46d6401aab1dde7c7de23dc55c037
10952317,7b48a87effd31c9c07b68ed212062854,61c46d6401aab1dde7c7de23dc55c037,7e54c9199aad70e35fe256d23701bad0
10952318,61c46d6401aab1dde7c7de23dc55c037,7e54c9199aad70e35fe256d23701bad0,6178580fa01b62e9b52787902c0d8ae6
10952319,7e54c9199aad70e35fe256d23701bad0,6178580fa01b62e9b52787902c0d8ae6,ab694649c65477d0bc574bf391a3f4a0
10952320,6178580fa01b62e9b52787902c0d8ae6,ab694649c65477d0bc574bf391a3f4a0,5b3387fa195672dcfe979d17e4a62c9e
10952321,ab694649c65477d0bc574bf391a3f4a0,5b3387fa195672dcfe979d17e4a62c9e,2790c612d8d301e2f35550c75aea8c75
10952322,5b3387fa195672dcfe979d17e4a62c9e,2790c612d8d301e2f35550c75aea8c75,d36c6cf30154e18e6c972704206d6b1e
10952323,2790c612d8d301e2f35550c75aea8c75,d36c6cf30154e18e6c972704206d6b1e,1cbcc681ecf7acef4948bff2eb8e39d7
10952324,d36c6cf30154e18e6c972704206d6b1e,1cbcc681ecf7acef4948bff2eb8e39d7,95eb6b55a0b6d049aadd729aaabd63de
10952325,1cbcc681ecf7acef4948bff2eb8e39d7,95eb6b55a0b6d049aadd729aaabd63de,7035839edc259dcad4b1632d10eded74


### Inference


Get testsource trainX

In [153]:
one_gram_train_data = getTrainData(train_source, 1)
two_gram_train_data = getTrainData(train_source, 2)
one_gram_test_data = getTrainData(test_source, 1)
two_gram_test_data = getTrainData(test_source, 2)

one_gramData = pd.concat(
    [one_gram_train_data, one_gram_test_data], axis=0, ignore_index=True
)
two_gramData = pd.concat(
    [two_gram_train_data, two_gram_test_data], axis=0, ignore_index=True
)

# Freq of ngram
twoGramFreq = getFreq(two_gramData, 2)
oneGramFreq = getFreq(one_gramData, 1)

twoGramFreq

,song_id,next1_song_id,next2_song_id,freq
10095227,ffffec8b691fb58debb2587b0e48c71f,b5a3cf61efaf108370719d1c2e5edd25,94ac23554609d19d2a5d9b1163e75f98,1.000000
10095226,ffffe1b0071c87b3716f708f7066a22f,949ad11fe2f6e6fbf6228bbffb9b36fb,599f2b612dfe484625d686b69e772272,1.000000
10095225,ffffcf62a064a729b55618a2648c6af0,d9779ba8a5733c542b644413164a3c0e,ece2c29d573c996c07199ade20a8d09b,1.000000
10095224,ffffcf62a064a729b55618a2648c6af0,2da05fbed3e68d10ccf6c0939d7e52de,ece2c29d573c996c07199ade20a8d09b,1.000000
10095223,ffffcf62a064a729b55618a2648c6af0,29161155b8d60571e4948243ca29b98d,4eec9ff2cec05b9e978a9c202fc103e1,1.000000
...,...,...,...,...
5,00008cfd4308f6abae6136217bc95e32,0df8c3fe1bbac6f81fa8c98580a618d6,00008cfd4308f6abae6136217bc95e32,1.000000
4,00005d588ff5b96c84556f17b716e227,4cfbb7a6084c21c3565b4b69065c3fdb,b3a64d0705689d65f402676999b6d3ed,1.000000
3,00002d2ddb6c789bcb8ef14f52339e39,bc74c4a61822fd7bc58e55a8b4159ae1,689333702ad97af27e22b40ba11614e5,1.000000
2,00002d2ddb6c789bcb8ef14f52339e39,7ae025da34fbb414d9beaa5bb5fef66e,1b8b6d30519419eb7b272fac7cffdfd4,1.000000


In [219]:
oneGramFreq

,song_id,next1_song_id,freq
8674264,ffffec8b691fb58debb2587b0e48c71f,b5a3cf61efaf108370719d1c2e5edd25,1.00
8674263,ffffe1b0071c87b3716f708f7066a22f,949ad11fe2f6e6fbf6228bbffb9b36fb,1.00
8674259,ffffcf62a064a729b55618a2648c6af0,d9779ba8a5733c542b644413164a3c0e,0.25
8674257,ffffc0ee02f3fac2b4a10ed7a6c3423f,910a57d453ff343c41019b2f28f63cb2,0.50
8674256,ffffb17f494477ddef244df20d662e01,5cac7624828ccdcbed56e3620fc9d6d5,1.00
...,...,...,...
7,0000a2175753bc165e53c408589a3bd6,4e001cbaae085dc4dacf707a86984452,1.00
5,00008cfd4308f6abae6136217bc95e32,0df8c3fe1bbac6f81fa8c98580a618d6,0.50
4,00005d588ff5b96c84556f17b716e227,4cfbb7a6084c21c3565b4b69065c3fdb,1.00
3,000049836b2e92a6a361e40d7f0c73db,32e10870dd5265133045e51b3910c905,1.00


In [166]:
def getTestX(df, n=2):
    trainX = pd.DataFrame()
    # if n == 3:, get listening_order : 18, 19, 20
    for i in range(n, 0, -1):
        _ = df.query(f"listening_order == {20-i+1}")[
            ["session_id", "song_id"]
        ].set_index("session_id")
        if trainX.empty:
            trainX = _
        else:
            trainX = trainX.join(_, lsuffix=f"_{n-i-1}")

    trainX = trainX.rename(columns={trainX.columns[-1]: f"song_id_{n-1}"})

    # trainX keep session_id
    trainX.reset_index(inplace=True)
    trainX.rename_axis("session_id", axis=1, inplace=True)
    return trainX


testX = getTestX(
    test_source,
)
testX.head(20)

session_id,session_id,song_id_0,song_id_1
0,598,8cd8008ddffd9b2d12b6b7bf17474e9e,e8bbc37dee09b8e548f0e2137dd1c2a5
1,1039,1ac480c8add024f6febd936dac34bf82,308f5f2b339b3ff74c9b60b49832d27f
2,1199,3aa676e51e8958a78394f6b6a07c2cea,d1492ba4a2fdf555510934e9740ea8c3
3,1489,e960bf0e27715aeae5620e6c60dead5f,4c19e9de0a90902ccd3b55374942fc5d
4,1868,337f6da1c3f672d989537305209c5347,4a12c85cadde3aaee76be944624a3516
5,2069,56a92b9699baff3512550b368f05ab3d,42b4d47845557afb0a404465258ba6bf
6,2659,d0f29a768191d3f75a4722789793db64,d0f29a768191d3f75a4722789793db64
7,2978,456c4d99c8403be96d52bd665add69ca,456c4d99c8403be96d52bd665add69ca
8,3259,34e874027e8fe62f3dccf84c74891f3a,34e874027e8fe62f3dccf84c74891f3a
9,5519,5b23ba9d45ab907d40bdaa736358f6b5,f75a3765b489834424dd242cd2de585c


Predict n gram's next column: song 21 

In [246]:
twoGramResult = pd.merge(
    testX,
    twoGramFreq,
    how="left",
    left_on=["song_id_0", "song_id_1"],
    right_on=["song_id", "next1_song_id"],
    suffixes=("_t", "_p"),  # Add suffixes to distinguish between columns
)
del twoGramResult["song_id"]
del twoGramResult["next1_song_id"]
twoGramResult

,session_id,song_id_0,song_id_1,next2_song_id,freq
0,598,8cd8008ddffd9b2d12b6b7bf17474e9e,e8bbc37dee09b8e548f0e2137dd1c2a5,aad06498fcd9699bee8bfb6bd9181943,0.153846
1,1039,1ac480c8add024f6febd936dac34bf82,308f5f2b339b3ff74c9b60b49832d27f,NaN,NaN
2,1199,3aa676e51e8958a78394f6b6a07c2cea,d1492ba4a2fdf555510934e9740ea8c3,NaN,NaN
3,1489,e960bf0e27715aeae5620e6c60dead5f,4c19e9de0a90902ccd3b55374942fc5d,NaN,NaN
4,1868,337f6da1c3f672d989537305209c5347,4a12c85cadde3aaee76be944624a3516,269ef1b3c7933d8de1d8dcb834630a4c,0.333333
...,...,...,...,...,...
143059,621339,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24,0.998353
143060,317578,7e1b53063809dc95d5282b4617e73105,1f604ad500399bfeba17734d0881673c,NaN,NaN
143061,712209,506e7dfd72ce5e3372c85d8246212b61,6694c67088568828cc3c10340dc9614e,NaN,NaN
143062,713779,70e49ffdfc75a802ea4d03ecdc7d66d9,8c269e6fb03a46d4e6e60b92e8e18771,3a756f3040d7d3e6aadebc714bfcd29e,1.000000


In [247]:
one_gram_result = pd.merge(
    twoGramResult,
    oneGramFreq,
    how="left",
    left_on=["song_id_1"],
    right_on=["song_id"],
    suffixes=("_t", "_p"),  # Add suffixes to distinguish between columns
)
del one_gram_result["song_id"]

In [212]:
one_gram_result

,session_id,song_id_0,song_id_1,next2_song_id,freq_t,next1_song_id,freq_p
0,598,8cd8008ddffd9b2d12b6b7bf17474e9e,e8bbc37dee09b8e548f0e2137dd1c2a5,aad06498fcd9699bee8bfb6bd9181943,0.153846,e8bbc37dee09b8e548f0e2137dd1c2a5,0.049930
1,1039,1ac480c8add024f6febd936dac34bf82,308f5f2b339b3ff74c9b60b49832d27f,NaN,NaN,ef587582918ea93602e08c29ffeb47c0,0.357143
2,1199,3aa676e51e8958a78394f6b6a07c2cea,d1492ba4a2fdf555510934e9740ea8c3,NaN,NaN,d1492ba4a2fdf555510934e9740ea8c3,0.097738
3,1489,e960bf0e27715aeae5620e6c60dead5f,4c19e9de0a90902ccd3b55374942fc5d,NaN,NaN,6cfb851242760855d0c85c9ca089c8c5,0.500000
4,1868,337f6da1c3f672d989537305209c5347,4a12c85cadde3aaee76be944624a3516,269ef1b3c7933d8de1d8dcb834630a4c,0.333333,337f6da1c3f672d989537305209c5347,0.069952
...,...,...,...,...,...,...,...
143059,621339,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24,0.998353,d3b629c521230e2b730b80ac65a40b24,0.980183
143060,317578,7e1b53063809dc95d5282b4617e73105,1f604ad500399bfeba17734d0881673c,NaN,NaN,NaN,NaN
143061,712209,506e7dfd72ce5e3372c85d8246212b61,6694c67088568828cc3c10340dc9614e,NaN,NaN,7d6c4e4d9b31d3994b4fdd1cde63bb4a,0.106061
143062,713779,70e49ffdfc75a802ea4d03ecdc7d66d9,8c269e6fb03a46d4e6e60b92e8e18771,3a756f3040d7d3e6aadebc714bfcd29e,1.000000,3a756f3040d7d3e6aadebc714bfcd29e,1.000000


In [249]:
has_nan = one_gram_result["next2_song_id"].isna().sum()
print(has_nan / one_gram_result.shape[0])

0.5692627075993961


In [244]:
# fillna with song_id_1
one_gram_result["next2_song_id"].fillna(one_gram_result["next1_song_id"], inplace=True)
one_gram_result.rename(columns={"next2_song_id": "top1"}, inplace=True)

# Check if "next2_song_id" column has any NaN values
# has_nan = one_gram_result["top1"].isna().any()
has_nan = one_gram_result["top1"].isna().sum()
print(has_nan)


# # Print the result
# if has_nan:
#     one_gram_result["top1"].fillna(one_gram_result["song_id_1"], inplace=True)

# else:
#     print("Column 'top1' does not have any NaN values.")


one_gram_result.head()

3176


,session_id,song_id_0,song_id_1,top1,freq_t,next1_song_id,freq_p
0,598,8cd8008ddffd9b2d12b6b7bf17474e9e,e8bbc37dee09b8e548f0e2137dd1c2a5,aad06498fcd9699bee8bfb6bd9181943,0.153846,e8bbc37dee09b8e548f0e2137dd1c2a5,0.049930
1,1039,1ac480c8add024f6febd936dac34bf82,308f5f2b339b3ff74c9b60b49832d27f,ef587582918ea93602e08c29ffeb47c0,NaN,ef587582918ea93602e08c29ffeb47c0,0.357143
2,1199,3aa676e51e8958a78394f6b6a07c2cea,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,NaN,d1492ba4a2fdf555510934e9740ea8c3,0.097738
3,1489,e960bf0e27715aeae5620e6c60dead5f,4c19e9de0a90902ccd3b55374942fc5d,6cfb851242760855d0c85c9ca089c8c5,NaN,6cfb851242760855d0c85c9ca089c8c5,0.500000
4,1868,337f6da1c3f672d989537305209c5347,4a12c85cadde3aaee76be944624a3516,269ef1b3c7933d8de1d8dcb834630a4c,0.333333,337f6da1c3f672d989537305209c5347,0.069952


In [215]:
one_gram_result.drop(
    columns=["song_id_0", "song_id_1", "next1_song_id", "freq_t", "freq_p"],
    inplace=True,
)
one_gram_result.head()

,session_id,top1
0,598,aad06498fcd9699bee8bfb6bd9181943
1,1039,ef587582918ea93602e08c29ffeb47c0
2,1199,d1492ba4a2fdf555510934e9740ea8c3
3,1489,6cfb851242760855d0c85c9ca089c8c5
4,1868,269ef1b3c7933d8de1d8dcb834630a4c


In [217]:
for i in range(2, 6):
    new_col_name = f"top{i}"
    one_gram_result[new_col_name] = one_gram_result["top1"]
one_gram_result

,session_id,top1,top2,top3,top4,top5
0,598,aad06498fcd9699bee8bfb6bd9181943,aad06498fcd9699bee8bfb6bd9181943,aad06498fcd9699bee8bfb6bd9181943,aad06498fcd9699bee8bfb6bd9181943,aad06498fcd9699bee8bfb6bd9181943
1,1039,ef587582918ea93602e08c29ffeb47c0,ef587582918ea93602e08c29ffeb47c0,ef587582918ea93602e08c29ffeb47c0,ef587582918ea93602e08c29ffeb47c0,ef587582918ea93602e08c29ffeb47c0
2,1199,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3
3,1489,6cfb851242760855d0c85c9ca089c8c5,6cfb851242760855d0c85c9ca089c8c5,6cfb851242760855d0c85c9ca089c8c5,6cfb851242760855d0c85c9ca089c8c5,6cfb851242760855d0c85c9ca089c8c5
4,1868,269ef1b3c7933d8de1d8dcb834630a4c,269ef1b3c7933d8de1d8dcb834630a4c,269ef1b3c7933d8de1d8dcb834630a4c,269ef1b3c7933d8de1d8dcb834630a4c,269ef1b3c7933d8de1d8dcb834630a4c
...,...,...,...,...,...,...
143059,621339,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24
143060,317578,1f604ad500399bfeba17734d0881673c,1f604ad500399bfeba17734d0881673c,1f604ad500399bfeba17734d0881673c,1f604ad500399bfeba17734d0881673c,1f604ad500399bfeba17734d0881673c
143061,712209,7d6c4e4d9b31d3994b4fdd1cde63bb4a,7d6c4e4d9b31d3994b4fdd1cde63bb4a,7d6c4e4d9b31d3994b4fdd1cde63bb4a,7d6c4e4d9b31d3994b4fdd1cde63bb4a,7d6c4e4d9b31d3994b4fdd1cde63bb4a
143062,713779,3a756f3040d7d3e6aadebc714bfcd29e,3a756f3040d7d3e6aadebc714bfcd29e,3a756f3040d7d3e6aadebc714bfcd29e,3a756f3040d7d3e6aadebc714bfcd29e,3a756f3040d7d3e6aadebc714bfcd29e


In [218]:
one_gram_result.to_csv("results/result.csv", index=False)

only one gram

In [238]:
one_gram_result = pd.merge(
    testX,
    oneGramFreq,
    how="left",
    left_on=["song_id_1"],
    right_on=["song_id"],
    suffixes=("_t", "_p"),  # Add suffixes to distinguish between columns
)
del one_gram_result["song_id"]
one_gram_result

,session_id,song_id_0,song_id_1,next1_song_id,freq
0,598,8cd8008ddffd9b2d12b6b7bf17474e9e,e8bbc37dee09b8e548f0e2137dd1c2a5,e8bbc37dee09b8e548f0e2137dd1c2a5,0.049930
1,1039,1ac480c8add024f6febd936dac34bf82,308f5f2b339b3ff74c9b60b49832d27f,ef587582918ea93602e08c29ffeb47c0,0.357143
2,1199,3aa676e51e8958a78394f6b6a07c2cea,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,0.097738
3,1489,e960bf0e27715aeae5620e6c60dead5f,4c19e9de0a90902ccd3b55374942fc5d,6cfb851242760855d0c85c9ca089c8c5,0.500000
4,1868,337f6da1c3f672d989537305209c5347,4a12c85cadde3aaee76be944624a3516,337f6da1c3f672d989537305209c5347,0.069952
...,...,...,...,...,...
143059,621339,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24,0.980183
143060,317578,7e1b53063809dc95d5282b4617e73105,1f604ad500399bfeba17734d0881673c,NaN,NaN
143061,712209,506e7dfd72ce5e3372c85d8246212b61,6694c67088568828cc3c10340dc9614e,7d6c4e4d9b31d3994b4fdd1cde63bb4a,0.106061
143062,713779,70e49ffdfc75a802ea4d03ecdc7d66d9,8c269e6fb03a46d4e6e60b92e8e18771,3a756f3040d7d3e6aadebc714bfcd29e,1.000000


In [239]:
# fillna with song_id_1
one_gram_result["next1_song_id"].fillna(one_gram_result["song_id_1"], inplace=True)
one_gram_result.rename(columns={"next1_song_id": "top1"}, inplace=True)

# Check if "next2_song_id" column has any NaN values
has_nan = one_gram_result["top1"].isna().any()

# Print the result
if has_nan:
    print(has_nan)
    one_gram_result["top1"].fillna(one_gram_result["song_id_1"], inplace=True)
else:
    print("Column 'top1' does not have any NaN values.")

one_gram_result.head()

Column 'top1' does not have any NaN values.


,session_id,song_id_0,song_id_1,top1,freq
0,598,8cd8008ddffd9b2d12b6b7bf17474e9e,e8bbc37dee09b8e548f0e2137dd1c2a5,e8bbc37dee09b8e548f0e2137dd1c2a5,0.049930
1,1039,1ac480c8add024f6febd936dac34bf82,308f5f2b339b3ff74c9b60b49832d27f,ef587582918ea93602e08c29ffeb47c0,0.357143
2,1199,3aa676e51e8958a78394f6b6a07c2cea,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,0.097738
3,1489,e960bf0e27715aeae5620e6c60dead5f,4c19e9de0a90902ccd3b55374942fc5d,6cfb851242760855d0c85c9ca089c8c5,0.500000
4,1868,337f6da1c3f672d989537305209c5347,4a12c85cadde3aaee76be944624a3516,337f6da1c3f672d989537305209c5347,0.069952


In [240]:
one_gram_result.drop(columns=["song_id_0", "song_id_1", "freq"], inplace=True)
for i in range(2, 6):
    new_col_name = f"top{i}"
    one_gram_result[new_col_name] = one_gram_result["top1"]
one_gram_result.to_csv("results/predict0.csv", index=False)

In [241]:
one_gram_result

,session_id,top1,top2,top3,top4,top5
0,598,e8bbc37dee09b8e548f0e2137dd1c2a5,e8bbc37dee09b8e548f0e2137dd1c2a5,e8bbc37dee09b8e548f0e2137dd1c2a5,e8bbc37dee09b8e548f0e2137dd1c2a5,e8bbc37dee09b8e548f0e2137dd1c2a5
1,1039,ef587582918ea93602e08c29ffeb47c0,ef587582918ea93602e08c29ffeb47c0,ef587582918ea93602e08c29ffeb47c0,ef587582918ea93602e08c29ffeb47c0,ef587582918ea93602e08c29ffeb47c0
2,1199,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3,d1492ba4a2fdf555510934e9740ea8c3
3,1489,6cfb851242760855d0c85c9ca089c8c5,6cfb851242760855d0c85c9ca089c8c5,6cfb851242760855d0c85c9ca089c8c5,6cfb851242760855d0c85c9ca089c8c5,6cfb851242760855d0c85c9ca089c8c5
4,1868,337f6da1c3f672d989537305209c5347,337f6da1c3f672d989537305209c5347,337f6da1c3f672d989537305209c5347,337f6da1c3f672d989537305209c5347,337f6da1c3f672d989537305209c5347
...,...,...,...,...,...,...
143059,621339,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24,d3b629c521230e2b730b80ac65a40b24
143060,317578,1f604ad500399bfeba17734d0881673c,1f604ad500399bfeba17734d0881673c,1f604ad500399bfeba17734d0881673c,1f604ad500399bfeba17734d0881673c,1f604ad500399bfeba17734d0881673c
143061,712209,7d6c4e4d9b31d3994b4fdd1cde63bb4a,7d6c4e4d9b31d3994b4fdd1cde63bb4a,7d6c4e4d9b31d3994b4fdd1cde63bb4a,7d6c4e4d9b31d3994b4fdd1cde63bb4a,7d6c4e4d9b31d3994b4fdd1cde63bb4a
143062,713779,3a756f3040d7d3e6aadebc714bfcd29e,3a756f3040d7d3e6aadebc714bfcd29e,3a756f3040d7d3e6aadebc714bfcd29e,3a756f3040d7d3e6aadebc714bfcd29e,3a756f3040d7d3e6aadebc714bfcd29e
